# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [5]:
def summarize_results(patients):
    """(list) -> dict
    Process patient data from a pipe-delimited file and return a list of dictionaries.
    """
    result = []
    occurrence_count = {}

    for patient in patients:
        for patient_number, data in patient.items():
            [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address] = data
            risk = priest(sex, int(age), int(breath), int(float(o2sat) * 100), int(heart), int(systolic), float(temp), alertness, inspired, status)

            if risk < 0.01:
                hospital = 'Home'
                address = 'Home'

            patient_data = {
                patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
            }

            result.append(patient_data)



            for data_dict in result:
                if len(data_dict) == 1:
                    value_list = list(data_dict.values())[0]
                    if len(value_list) >= 11:
                        element_11 = value_list[10]
                        occurrence_count[element_11] = occurrence_count.get(element_11, 0) + 1




    return occurrence_count

In [6]:
print(summarize_results(process_people("https://hds5210-data.s3.amazonaws.com/people.psv")))

NameError: ignored